In [1]:
import json
from collections import Counter
from pathlib import Path
import pandas as pd

dataset_root = "INSERT MARYLAND POLYVORE ROOT"
train_filename = "train_no_dup.json"
valid_filename = "valid_no_dup.json"
test_filename = "test_no_dup.json"

counts = []

def process_dataset(dataset_filename):
    outfits_disposed = 0
    total_items = 0
    total_outfits = 0
    categories = []
    with open(Path(dataset_root, dataset_filename)) as json_file:
        raw_json = json.load(json_file)
        print("Loaded " + str(len(raw_json)) + " items", flush=True)
        examples = []
        for outfit in raw_json:
            images = []
            counts.append(len(outfit["items"]))
            for item in outfit["items"]:
                total_items = total_items + 1
                categories.append(item["categoryid"])
                images.append(1)
            if len(images) < 3:
                outfits_disposed = outfits_disposed + 1
                continue
            
            total_outfits = total_outfits + 1


    print("Total " + str(total_items) + " products")
    print("Total " + str(total_outfits) + " outfits")
    return examples

process_dataset(train_filename)
process_dataset(valid_filename)
process_dataset(test_filename)

counter = Counter(counts)
series = pd.Series(counts)
print(series.describe())
print(counter)

In [2]:
import random
import json


def create_FITB(dataset_filename):
    total_items = 0
    total_outfits = 0
    samples = []
    with open(Path(dataset_root, dataset_filename)) as json_file:
        raw_json = json.load(json_file)
        print("Loaded " + str(len(raw_json)) + " items", flush=True)
        
        for outfit in raw_json:
            sample = {
                "question" : [],
                "answers" : []
            }
            
            for item in outfit["items"]:
                total_items = total_items + 1
                sample["question"].append(outfit["set_id"] + "_" + str(item["index"]))
            
            # Draw random item form the outfit and put it to questions
            sample["blank_position"] = random.randint(1, len(sample["question"]))
            sample["answers"].append(sample["question"][sample["blank_position"] - 1])
            sample["question"].remove(sample["question"][sample["blank_position"] - 1])
            
            # Add 3 more answers
            for i in range(3):
                r_outfit = random.choice(raw_json)
                while r_outfit["set_id"] == outfit["set_id"]:
                    r_outfit = random.choice(raw_json)
                r_item = random.choice(r_outfit["items"])
                sample["answers"].append(r_outfit["set_id"] + "_" + str(r_item["index"]))
            
            total_outfits = total_outfits + 1
            samples.append(sample)
            
    with open('fill_in_blank_valid.json', 'w') as outfile:
        json.dump(samples, outfile)
    print("Total " + str(total_items) + " products")
    print("Total " + str(total_outfits) + " outfits")

# create_FITB("valid_no_dup.json")

Loaded 1497 items
Total 9070 products
Total 1497 outfits


In [ ]:
def search_unique_items(dataset_filename):
    with open(Path(dataset_root, dataset_filename)) as json_file:
        raw_json = json.load(json_file)
        print("Loaded " + str(len(raw_json)) + " outfits", flush=True)
        unique_ids = {}
        duplicates = {}
        total_items = 0
        outfits_without_unique = 0
        for outfit in raw_json:
            for item in outfit["items"]:
                total_items = total_items + 1
                _, id = item['image'].split('id=')
                if id not in unique_ids:
                    unique_ids[id] = True
                else:
                    duplicates[id] = True
                    
        for outfit in raw_json:
            for item in outfit["items"]:
                _, id = item['image'].split('id=')
                if id in duplicates:
                    outfits_without_unique += 1
                    
        print("Total " + str(total_items))
        print("Unique " + str(len(unique_ids)))
        print("outfits_without_unique " + str(outfits_without_unique))